In [35]:
# https://python.langchain.com/docs/integrations/document_loaders/web_base/
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [36]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [37]:
loader = WebBaseLoader("https://raw.githubusercontent.com/Emmerson-Miranda/prompts/refs/heads/main/user_stories/user_stories_generated.md")
pages = loader.load_and_split()

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

chunks = text_splitter.split_documents(pages)

In [40]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [41]:
vector_store = InMemoryVectorStore.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever()

In [42]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [43]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [44]:
template = """You are a helpful AI assistant. 
Be factual in your response.
Respond the folowing question: {question} only from below context: {context}.
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

prompt = PromptTemplate.from_template(template)

chain = (
  {"context": retriever | format_docs, "question": RunnablePassthrough()}
  | prompt  | llm | StrOutputParser()
)

In [45]:
chain.invoke( "What is the first user story?")

'The first user story is "User Authentication" which states: As a Google user, I want to log in using my Google account, so that my identity is authenticated securely and my progress can be tracked.'

In [46]:
chain.invoke( "Which credentials I have to use to login?")

'To login, you can use your Google account credentials.'

In [51]:
chain.invoke( "people with disabilities can play the game?")

'Yes, people with disabilities can play the game as long as the application meets WCAG 2.2 Level A compliance for accessibility.'

In [52]:
chain.invoke( "summarize the document")

'The document contains high-level user stories for the Tic-Tac-Toe Web Application project, including requirements for localization, accessibility, deployment to AWS, and privacy disclosure. These user stories are structured in the Cucumber format and adhere to SCRUM guidelines.'